# Assignment 2 - Randomized Optimization - Part II
## student: hyao66

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mlrose

In [ ]:
!pip install mlrose-hiive

In [ ]:
# import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import six
import sys
sys.modules['sklearn.externals.six'] = six

import mlrose
import mlrose_hiive
import time

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn import metrics
# from yellowbrick.model_selection import LearningCurve, learning_curve
from sklearn.model_selection import StratifiedKFold
import seaborn as sns

## part II: apply gd, rhc, sa and ga to NN

In [ ]:
## load and process data

raw = pd.read_csv('/content/drive/MyDrive/CS7641A2/customer_segmentation.csv', index_col = 'ID')

# label encoding for categorical data
raw['Gender'].astype('category')
raw['Ever_Married'].astype('category')
raw['Graduated'].astype('category')
raw['Profession'].astype('category')
raw['Spending_Score'].astype('category')
raw['Var_1'].astype('category')
raw['Segmentation'].astype('category')
le = preprocessing.LabelEncoder()
raw['Gender_Cat'] = le.fit_transform(raw['Gender'])
raw['Ever_Married_Cat'] = le.fit_transform(raw['Ever_Married'])
raw['Graduated_Cat'] = le.fit_transform(raw['Graduated'])
raw['Profession_Cat'] = le.fit_transform(raw['Profession'])
raw['Spending_Score_Cat'] = le.fit_transform(raw['Spending_Score'])
raw['Var_1_Cat'] = le.fit_transform(raw['Var_1'])
raw['Segmentation_Cat'] = le.fit_transform(raw['Segmentation'])

raw = raw.dropna()


# train test split
# split data into training and testing sets
# Split the data into training and testing sets
raw_cat = raw.drop(["Gender","Ever_Married","Graduated","Profession","Spending_Score","Var_1","Segmentation"], axis = 1)


features = raw_cat.drop(['Segmentation_Cat'], axis = 1)
feature_list = list(features.columns)
features_np = np.array(features)

labels = raw_cat[['Segmentation_Cat']]
labels_np = np.array(labels)[:,0]

label_unique = list(set(labels_np))

train_features, test_features, train_labels, test_labels = train_test_split(features_np, labels_np, test_size = 0.20, 
                                                                            random_state = 42)

print(feature_list)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


# One hot encode target values
one_hot = OneHotEncoder()

train_labels_onehot = one_hot.fit_transform(train_labels.reshape(-1, 1)).todense()
test_labels_onehot = one_hot.transform(test_labels.reshape(-1, 1)).todense()


label = LabelEncoder()
train_labels_label = label.fit_transform(train_labels)
test_labels_label = label.fit_transform(test_labels)

# normalize x
scaler = MinMaxScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [ ]:
##########
#Back Prop
##########
clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                           algorithm = 'gradient_descent',early_stopping = False, 
                           max_attempts = 100, max_iters = 2500, clip_max = 5,
                           bias = True, learning_rate = 0.00001,
                           curve = True, random_state=42)

start_time = time.time()
clf.fit(train_features, train_labels_onehot)
end_time = time.time()
base_time = end_time-start_time
y_pred = clf.predict(test_features)
gd_score = metrics.accuracy_score(test_labels_onehot, y_pred)
gd_fitness_curve = clf.fitness_curve

print('train time:', base_time)
print('accuracy:', gd_score)

In [ ]:
gd_fitness_curve

In [ ]:
print('train time:', base_time)
print('accuracy:', gd_score)

plt.figure()

plt.plot(gd_fitness_curve)
plt.xlabel("Iterations")
plt.ylabel("Negative Loss")
plt.title('Fitness curve for NN with gradient descent')

In [ ]:

gd_fitness_curve = [ -x for x in gd_fitness_curve]

plt.figure()

plt.plot(gd_fitness_curve)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title('Loss curve for NN with gradient descent')

In [ ]:
results = []
for i in range(1, 2601, 100):
        clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                                   algorithm ='gradient_descent',early_stopping = False, 
                                   max_attempts = 100, max_iters = i, clip_max = 5,
                                   bias = True, learning_rate = 0.00001,
                                   curve = False, random_state=42)
        clf.fit(train_features, train_labels_onehot)
        y_train_pred = clf.predict(train_features)
        y_train_accuracy = metrics.accuracy_score(train_labels_onehot, y_train_pred)

        y_pred = clf.predict(test_features)
        y_test_accuracy = metrics.accuracy_score(test_labels_onehot, y_pred)

        results.append([i, 'gradient_descent', y_train_accuracy, y_test_accuracy])
        print([i, 'gradient_descent', y_train_accuracy, y_test_accuracy])
        

In [ ]:
# plot learning curve
df = pd.DataFrame(results, columns=["Iterations", "Algorithm", "Train_Accuracy", "Test_Accuracy"])

In [ ]:
plt.figure()
plt.plot(df['Iterations'], df['Train_Accuracy'], marker='o')
plt.ylabel('Accuracy')
plt.title('Neural Net experiments on Gradient Descent')
plt.plot(df['Iterations'], df['Test_Accuracy'], marker='d')
plt.xlabel('Iterations')
plt.legend(['train', 'test'])

In [ ]:
##########
#RHC
##########

clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                           algorithm = 'random_hill_climb',early_stopping = False, 
                           max_attempts = 100, max_iters = 2500, clip_max = 5,
                           bias = True, learning_rate = 0.1,
                           curve = True, random_state=42)
start_time = time.time()
clf.fit(train_features, train_labels_onehot)
end_time = time.time()
rhc_time = end_time-start_time
y_pred = clf.predict(test_features)
rhc_score = metrics.accuracy_score(test_labels_onehot, y_pred)
rhc_fitness_curve = clf.fitness_curve

print('train time:', rhc_time)
print('accuracy:', rhc_score)



In [ ]:
##########
#RHC
##########

clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                           algorithm = 'random_hill_climb',early_stopping = False, 
                           max_attempts = 1000000, max_iters = 1, clip_max = 5,
                           bias = True, learning_rate = 1,
                           curve = True, random_state=42)
start_time = time.time()
clf.fit(train_features, train_labels_onehot)
end_time = time.time()
rhc_time = end_time-start_time
y_pred = clf.predict(test_features)
rhc_score = metrics.accuracy_score(test_labels_onehot, y_pred)
rhc_fitness_curve = clf.fitness_curve

print('train time:', rhc_time)
print('accuracy:', rhc_score)

In [ ]:
rhc_fitness_curve = [ -x for x in rhc_fitness_curve]

plt.figure()

plt.plot(rhc_fitness_curve)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title('Loss curve for NN with RHC')

In [ ]:
rhc_fitness_curve[-10:]

In [ ]:
results = []

for i in range(1, 2601, 100):
        clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                                   algorithm ='random_hill_climb',early_stopping = False, 
                                   max_attempts = 100, max_iters = i, clip_max = 5,
                                   bias = True, learning_rate = 0.1,
                                   curve = False, random_state=42)
        clf.fit(train_features, train_labels_onehot)
        y_train_pred = clf.predict(train_features)
        y_train_accuracy = metrics.accuracy_score(train_labels_onehot, y_train_pred)

        y_pred = clf.predict(test_features)
        y_test_accuracy = metrics.accuracy_score(test_labels_onehot, y_pred)

        results.append([i, 'random_hill_climb', y_train_accuracy, y_test_accuracy])
        print([i, 'random_hill_climb', y_train_accuracy, y_test_accuracy])

In [ ]:
# plot learning curve
df = pd.DataFrame(results, columns=["Iterations", "Algorithm", "Train_Accuracy", "Test_Accuracy"])
plt.figure()
plt.plot(df['Iterations'], df['Train_Accuracy'], marker='o')
plt.ylabel('Accuracy')
plt.title('Neural Net experiments on Random hill climbing')
plt.plot(df['Iterations'], df['Test_Accuracy'], marker='d')
plt.xlabel('Iterations')
plt.legend(['train', 'test'])

In [ ]:
##########
#SA
##########

clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                           algorithm = 'simulated_annealing',early_stopping = False, 
                           max_attempts = 100, max_iters = 2500, clip_max = 5,
                           bias = True, learning_rate = 0.1,schedule=mlrose.GeomDecay(init_temp = 100, decay=0.1, min_temp=1),
                           curve = True, random_state=1)
start_time = time.time()
clf.fit(train_features, train_labels_onehot)
end_time = time.time()
sa_time = end_time-start_time
y_pred = clf.predict(test_features)
sa_score = metrics.accuracy_score(test_labels_onehot, y_pred)
sa_fitness_curve = clf.fitness_curve

print('train time:', sa_time)
print('accuracy:', sa_score)



In [ ]:
sa_fitness_curve = [ -x for x in sa_fitness_curve]

plt.figure()

plt.plot(sa_fitness_curve)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title('Loss curve for NN with SA')

In [ ]:
results = []


for i in range(1, 2601, 100):
        clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                                   algorithm ='simulated_annealing',early_stopping = False, 
                                   max_attempts = 100, max_iters = i, clip_max = 5,
                                   bias = True, learning_rate = 0.1,schedule=mlrose.GeomDecay(init_temp = 100, decay=0.1, min_temp=1),
                                   curve = False, random_state=1)
        clf.fit(train_features, train_labels_onehot)
        y_train_pred = clf.predict(train_features)
        y_train_accuracy = metrics.accuracy_score(train_labels_onehot, y_train_pred)

        y_pred = clf.predict(test_features)
        y_test_accuracy = metrics.accuracy_score(test_labels_onehot, y_pred)

        results.append([i, 'simulated_annealing', y_train_accuracy, y_test_accuracy])
        print([i, 'simulated_annealing', y_train_accuracy, y_test_accuracy])

In [ ]:
# plot learning curve
df = pd.DataFrame(results, columns=["Iterations", "Algorithm", "Train_Accuracy", "Test_Accuracy"])
plt.figure()
plt.plot(df['Iterations'], df['Train_Accuracy'], marker='o')
plt.ylabel('Accuracy')
plt.title('Neural Net experiments on simulated annealing')
plt.plot(df['Iterations'], df['Test_Accuracy'], marker='d')
plt.xlabel('Iterations')
plt.legend(['train', 'test'])

In [ ]:
## tune GA 
results = []

# max_attempts = 100
max_iters = 5

max_attempts_list = [100, 500, 1000, 5000]
pop_size_list = [100, 200, 500]
mutation_prob_list = [0.001, 0.005, 0.01, 0.05, 0.1]

# run generic algorithm
for pop in pop_size_list:
  for mutation in mutation_prob_list:
    for max_attempt in max_attempts_list:
        clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                                  algorithm = 'genetic_alg',early_stopping = False, 
                                  max_attempts = max_attempt, max_iters = max_iters, clip_max = 5,
                                  bias = True, learning_rate = 0.1, pop_size=pop, mutation_prob=mutation,
                                  curve = False, random_state=42)
        start_time = time.time()
        clf.fit(train_features, train_labels_onehot)
        end_time = time.time()
        ga_time = end_time-start_time
        y_pred = clf.predict(test_features)
        ga_score = metrics.accuracy_score(test_labels_onehot, y_pred)

        results.append([pop, mutation, max_attempt, 'GA', ga_score, ga_time])
        print([pop, mutation, max_attempt, ga_score, ga_time])

In [ ]:
##########
#GA
##########

clf = mlrose.NeuralNetwork(hidden_nodes=[100,100,50,50], activation='sigmoid',
                           algorithm = 'genetic_alg',early_stopping = False, 
                           max_attempts = 100, max_iters = 500, clip_max = 5,
                           bias = True, learning_rate = 0.1, pop_size=500, mutation_prob=0.001,
                           curve = True, random_state=42)
start_time = time.time()
clf.fit(train_features, train_labels_onehot)
end_time = time.time()
ga_time = end_time-start_time
y_pred = clf.predict(test_features)
ga_score = metrics.accuracy_score(test_labels_onehot, y_pred)
ga_fitness_curve = clf.fitness_curve

print('train time:', ga_time)
print('accuracy:', ga_score)

In [ ]:
ga_fitness_curve = [ -x for x in ga_fitness_curve]

plt.figure()

plt.plot(ga_fitness_curve)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title('Loss curve for NN with GA')

In [ ]:
ga_fitness_curve